In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from yahoofinancials import YahooFinancials

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [50]:
# Select stocks: Alibaba, Google,Netflix, Tesla, SP500, weekly,5 years
assets = ['BABA','AAPL', 'TSLA','NFLX']
yahoo_financials = YahooFinancials(assets)

data = yahoo_financials.get_historical_price_data(start_date = '2015-01-01',
                                                  end_date = '2020-11-04',
                                                  time_interval = 'weekly')


### Data Preperation: 4 stocks, weekly return, log return, then make as portoflio

In [51]:
#data frame these stocks
stocks = pd.DataFrame({
      a: {x['formatted_date']: x['adjclose'] for x in data[a]['prices']} for a in assets
})
stocks.index.names = ['Date'] # note the data columns are index. cuz in the dictionary: key: value. keys are indexes. 


In [52]:
# weekly return
stocks.pct_change(1).head()  #pct_change(periods)

,BABA,AAPL,TSLA,NFLX
Date,,,,
2015-01-01,NaN,NaN,NaN,NaN
2015-01-08,-0.024968,0.019026,-0.086561,-0.009046
2015-01-15,0.037256,-0.002277,0.020136,0.262275
2015-01-22,-0.046858,0.052579,0.014244,0.081069
2015-01-29,-0.085830,0.036857,0.096203,0.014126


In [23]:
# mean weekly return
stocks.pct_change(1).mean()

BABA    0.004467
AAPL    0.005590
TSLA    0.021861
NFLX    0.009396
dtype: float64

In [24]:
# calculate log return. why use log return:  reduces the variation of the time series making it easier to fit the model in question.
log_return = np.log(stocks/stocks.shift(1))
log_return.head()

,BABA,AAPL,TSLA,NFLX
Date,,,,
2015-01-01,NaN,NaN,NaN,NaN
2015-01-08,-0.025285,0.018847,-0.090538,-0.009088
2015-01-15,0.036579,-0.002279,0.019936,0.232916
2015-01-22,-0.047992,0.051243,0.014144,0.077951
2015-01-29,-0.089739,0.036194,0.091852,0.014027


In [25]:
log_return.mean()

BABA    0.003382
AAPL    0.004969
TSLA    0.007590
NFLX    0.007711
dtype: float64

In [38]:
#select 4 stocks for portofolio
portfolio = log_return.iloc[:,0:4]
portfolio.head()

,BABA,AAPL,TSLA,NFLX
Date,,,,
2015-01-01,NaN,NaN,NaN,NaN
2015-01-08,-0.025285,0.018847,-0.090538,-0.009088
2015-01-15,0.036579,-0.002279,0.019936,0.232916
2015-01-22,-0.047992,0.051243,0.014144,0.077951
2015-01-29,-0.089739,0.036194,0.091852,0.014027


###  Optimization Algorithm
#### minimize the negative Sharpe Ratio 

In [39]:
def get_ret_vol_sr(weights):
    weights = np.array(weights)
    ret = np.sum(portfolio.mean()*weights)
    vol = np.sqrt(np.dot(weights.T,np.dot(portfolio.cov(),weights)))
    sr = ret/vol
    return np.array([ret,vol,sr])

In [40]:
from scipy.optimize import minimize 

In [41]:
#minimize negative sharpe ratio
def neg_sharpe(weights):
    return get_ret_vol_sr(weights)[2] * -1

In [42]:
# add constriant:check allocation sums to 1
def check_sum(weights):
    return np.sum(weights) - 1

In [43]:
# create constraint variable: an equition type of constrant. fun: pass in the function: check_sum
cons = ({'type':'eq','fun':check_sum})

In [44]:
# create weight boundaries
bounds = ((0,1),(0,1),(0,1),(0,1))

In [45]:
# initial guess to start with
init_guess = [0.25, 0.25, 0.25, 0.25]

In [53]:
opt_results = minimize(neg_sharpe, init_guess, method='SLSQP', bounds=bounds, constraints=cons)

###### scipy.optimize.minimze(fun,x0,args = (),method = NONE, jac = NONE,bounds = , constraints =,tol = ,callback =, options = )
###### fun: the objective function to be minimized.
###### x0: ndarry,shape(n), initial guess
###### args: tuple, optional. 
###### method: str or callable. type of solver: SLSQP: minimize a scalar function of one or more variables using Sequential Least Sqaure Programming


In [47]:
opt_results

     fun: -0.16833145041596706
     jac: array([ 1.90625787e-02, -6.36130571e-05,  1.17090717e-03,  1.17102638e-04])
 message: 'Optimization terminated successfully'
    nfev: 40
     nit: 8
    njev: 8
  status: 0
 success: True
       x: array([0.        , 0.64798678, 0.        , 0.35201322])

In [48]:
#extract weights from results
opt_weights = opt_results.x

In [49]:
#apply weights to function to get return, volatiluty and opt_sharpe ratio

opt_ptf = pd.DataFrame(get_ret_vol_sr(opt_weights))
opt_ptf.index = ['Optimal Return','Optimal Volatility','Optimal Sharpe Ratio']
opt_ptf.columns =['Result']
opt_ptf

print('The optmal weights are: ', opt_weights)

,Result
Optimal Return,0.005934
Optimal Volatility,0.035254
Optimal Sharpe Ratio,0.168331


The optmal weights are:  [0.         0.64798678 0.         0.35201322]
